Load Trade Embeddings

In [1]:
import os
from langchain.storage import LocalFileStore
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from apikey import apikey
from langchain import OpenAI
from langchain.chains import RetrievalQA

os.environ['OPENAI_API_KEY'] = apikey

embeddings = OpenAIEmbeddings()

db = FAISS.load_local("faiss_index_trades", embeddings)

llm = OpenAI(temperature=0.6, openai_api_key=apikey)
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="refine", retriever=db.as_retriever())

/home/prash/.local/lib/python3.10/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (3.8.3) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In [13]:
from langchain.schema.runnable import RunnableMap, RunnablePassthrough
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnableMap, RunnablePassthrough
from operator import itemgetter

initialanalysis = (
    ChatPromptTemplate.from_template(
        "Keep the all the data and create an Explanation, a profit maximizing scenario, and trade instructions from this: {instructions}"
    )
    | ChatOpenAI(model_name="gpt-4", openai_api_key=apikey, verbose = True)
    | StrOutputParser()
    | {"base_response": RunnablePassthrough()}
)

explanation = (
    ChatPromptTemplate.from_messages(
        [
            ("ai", "{base_response}"),
            ("system", "Generate a correct explanation and make it more detailed"),
        ]
    )
    | ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_key=apikey, verbose = True)
    | StrOutputParser()
)

profit = (
    ChatPromptTemplate.from_messages(
        [
            ("ai", "{base_response}"),
            ("system", "Generate a correct profit maximizing scenario and make it more detailed"),
        ]
    )
    | ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_key=apikey, verbose = True)
    | StrOutputParser()
)

trade = (
    ChatPromptTemplate.from_messages(
        [
            ("ai", "{base_response}"),
            ("system", "Generate a correct set of steps with specific amounts to execute a $200 trade for this scenario"),
        ]
    )
    | ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_key=apikey, verbose = True)
    | StrOutputParser()
)

final_responder = (
    ChatPromptTemplate.from_messages(
        [
            ("ai", "{original_response}"),
            ("human", "ScenarioExplanation:\n{explanation}\n\n:\n{profit}\n\n:\n{trade}"),
            ("system", "Generate a final response including scenratio explanation, profit strategy, and trade actions. Explicitly state the input data set values"),
        ]
    )
    | ChatOpenAI(model_name="gpt-4", openai_api_key=apikey, verbose = True)
    | StrOutputParser()
)    

chain = (
    initialanalysis 
    | {
        "Trade": trade,
    }
)



In [14]:
datapoint = "ETH Price = 1621, BTC Price = 29450, Ratio = 0.055, Z-Score = -2.65"
chain.invoke({"instructions":str(datapoint)+"\n"+str(qa.run(datapoint))})

{'Trade': 'Trade Instructions:\n\n1. Sell $200 worth of BTC. Based on the current BTC price of $29,450, this equals approximately 0.0068 BTC.\n\n2. Use the $200 from the sale of BTC to buy ETH. Based on the current ETH price of $1621, this equals approximately 0.1234 ETH.\n\n3. Monitor the market and hold the ETH until the Z-Score is closer to 0, indicating that ETH has reached its average value relative to BTC.\n\n4. Sell the ETH for a profit.'}